In [ ]:
from pathlib import Path
from fastai.data.block import DataBlock
from fastai.data.transforms import RandomSplitter, get_files
from fastai.data.core import DataLoaders
from fastai.learner import Learner
from audiocrush.data import *
from fastai.data.block import CategoryBlock
from fastai.data.transforms import parent_label 
from audiocrush.models.resnet_model import ResNet
from audiocrush.models.models import *
import torch.nn.functional as F

from fastai.callback.schedule import fit_one_cycle
from fastai.losses import CrossEntropyLossFlat
from fastai.metrics import accuracy

from audiocrush.augment import *

In [ ]:
path = Path('C:/Data/Instruments')

fnames = get_audio_file_names(path)
fnames[0]
batch_tfms = [ # RandomNoise(),
               RandomResample() ]

dls = AudioDataLoaders.from_path_func(path, fnames, parent_label, valid_pct=0.2,
                                  seed=None, item_tfms=None,
                                  batch_tfms=batch_tfms)
dls.bs = 48

In [ ]:
dls.bs
print(dls.after_batch)

In [ ]:
print(f"DataLoaders device: {dls.device}")
print(dls.after_batch)
# dls.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"DataLoaders device: {dls.device}")
print(dls.after_batch)

In [ ]:
model = ConvModelSimple3regu(num_classes=4)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [ ]:
lr = learn.lr_find()

In [ ]:
# lr = 5e-6
lr # = 5e-3

In [ ]:
learn.fit_one_cycle(lr_max=lr, n_epoch=1)

In [ ]:
dls.show_batch()

In [ ]:
learn.fit_one_cycle(lr_max=lr, n_epoch=15)

In [ ]:
len(dls.valid_ds)

In [ ]:
from fastai.interpret import ClassificationInterpretation

interp = ClassificationInterpretation.from_learner(learn)


interp.plot_top_losses(k=10)
losses,idxs = interp.top_losses()

interp.vocab
interp.print_classification_report()
interp.top_losses(k=10)
print(interp.most_confused(min_val=5))

interp.plot_confusion_matrix(figsize=(12,12), dpi=60)


print(dls.valid_ds.items[36])
print(losses.shape,idxs.shape)
len(dls.valid_ds)==len(losses)==len(idxs)

dls.show_batch()

interp.show_results(idxs=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], nrows=2, ncols=5)


In [ ]:
interp.vocab

In [ ]:
xb, yb = dls.one_batch()
print(xb.shape, yb.shape)
print(xb, yb)

In [ ]:
preds, targs = learn.get_preds()
decoded_preds = preds.argmax(dim=1)

print(decoded_preds.shape, targs.shape) 
print(decoded_preds, targs) 

xb, yb = learn.dls.valid.one_batch()
print(xb.shape, yb.shape)

In [ ]:
learn.fit_one_cycle(lr_max=lr, n_epoch=1)

In [ ]:
learn.lr_find()

In [ ]:
lr = 2e-4
dls.bs = 128
print(dls.bs)

In [ ]:
learn.fit_one_cycle(lr_max=lr, n_epoch=500)

In [ ]:
learn.save('ConvModelSimple3regu-drums-02')

In [ ]:
print(learn.predict(Path("c:\\Data\\Special-validation\\fb\\snare-01.wav")))
print(learn.dls.vocab)

In [ ]:
a = (learn.get_preds(with_decoded=False))
# a = torch.softmax(a, dim=1)
# a = a.sum(dim=1)
for tensor in a:
    print(tensor.shape)


In [ ]:
print(learn.cbs)
print(learn.dls.vocab)
print(learn.dls.c)

In [ ]:
a = (learn.get_preds(with_decoded=True, with_input=True))
# a = torch.softmax(a, dim=1)
# a = a.sum(dim=1)
for tensor in a:
    print(tensor.shape)


In [ ]:
a, b = learn.get_preds()
# a, b = a.type(torch.FloatTensor), b.type(torch.LongTensor)
a.shape, b.shape

In [ ]:
from fastai.torch_core import flatten_check
flatten_check(a, b)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
preds, targs = a.argmax(dim=1), b  # If your predictions are logits
print(confusion_matrix(targs, preds))
print(classification_report(targs, preds))